### If there is a set of 10 positive numbers, are there two subsets that sum of the elements in both sets is equal?

#### objective function
$$minimize \sum_{i=0}^{10} n_{i}s_{i}$$

In [11]:
from pyqubo import Array
s=Array.create('s',shape=10,vartype='SPIN')   
H=sum(n*s for s,n in zip(s,range(10)))**2
model=H.compile()
qubo,offset=model.to_qubo()

In [12]:
bqm=model.to_dimod_bqm()

In [13]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
sampler=EmbeddingComposite(DWaveSampler())
response=sampler.sample(bqm)

In [14]:
for sample in response.samples():
     print(sample)

{'s[0]': 0, 's[1]': 0, 's[2]': 0, 's[3]': 0, 's[4]': 1, 's[5]': 1, 's[6]': 1, 's[7]': 0, 's[8]': 1, 's[9]': 1}


In [15]:
import pandas as pd
df=pd.DataFrame.from_dict(sample,orient='index').reset_index().reset_index()
ans={'group1':{'list':list(df[df[0]==0]['level_0']),'sum':sum(list(df[df[0]==0]['level_0']))},
    'group2:':{'list':list(df[df[0]==1]['level_0']),'sum':sum(list(df[df[0]==0]['level_0']))}}

In [16]:
print(pd.DataFrame.from_dict(ans))

               group1          group2:
list  [0, 1, 2, 3, 7]  [4, 5, 6, 8, 9]
sum                13               13


### Alternative answers

In [17]:
response=sampler.sample(bqm,num_reads=10)

In [18]:
for sample in response.samples():
    df=pd.DataFrame.from_dict(sample,orient='index').reset_index().reset_index()
    ans={'group1':{'list':list(df[df[0]==0]['level_0']),'sum':sum(list(df[df[0]==0]['level_0']))},'group2:':{'list':list(df[df[0]==1]['level_0']),'sum':sum(list(df[df[0]==0]['level_0']))}}
    print(pd.DataFrame.from_dict(ans))

               group1          group2:
list  [0, 1, 2, 7, 9]  [3, 4, 5, 6, 8]
sum                19               19
                  group1       group2:
list  [0, 1, 2, 7, 8, 9]  [3, 4, 5, 6]
sum                   27            27
            group1             group2:
list  [3, 7, 8, 9]  [0, 1, 2, 4, 5, 6]
sum             27                  27
               group1          group2:
list  [1, 3, 7, 8, 9]  [0, 2, 4, 5, 6]
sum                28               28
               group1          group2:
list  [0, 1, 2, 3, 8]  [4, 5, 6, 7, 9]
sum                14               14
               group1          group2:
list  [0, 1, 2, 3, 4]  [5, 6, 7, 8, 9]
sum                10               10
         group1                group2:
list  [1, 2, 3]  [0, 4, 5, 6, 7, 8, 9]
sum           6                      6
         group1                group2:
list  [0, 2, 4]  [1, 3, 5, 6, 7, 8, 9]
sum           6                      6
      group1                   group2:
list  [1, 2]  [0, 3, 4, 5

### Add constraint to balance the size of two sets

In [43]:
from pyqubo import Constraint
M=3
H=H+M*Constraint(sum(s for s in s)**2,label='(sum s)^2= 0')

In [44]:
model=H.compile()
qubo,offset=model.to_qubo()
bqm=model.to_dimod_bqm()

In [45]:
sampler=EmbeddingComposite(DWaveSampler())
response=sampler.sample(bqm,num_reads=5)

In [46]:
for sample in response.samples():
     print(sample)

{'s[0]': 0, 's[1]': 0, 's[2]': 0, 's[3]': 0, 's[4]': 0, 's[5]': 1, 's[6]': 1, 's[7]': 1, 's[8]': 1, 's[9]': 1}
{'s[0]': 0, 's[1]': 0, 's[2]': 0, 's[3]': 0, 's[4]': 0, 's[5]': 1, 's[6]': 1, 's[7]': 1, 's[8]': 1, 's[9]': 1}
{'s[0]': 0, 's[1]': 0, 's[2]': 0, 's[3]': 0, 's[4]': 0, 's[5]': 1, 's[6]': 1, 's[7]': 1, 's[8]': 1, 's[9]': 1}
{'s[0]': 0, 's[1]': 0, 's[2]': 0, 's[3]': 0, 's[4]': 0, 's[5]': 1, 's[6]': 1, 's[7]': 1, 's[8]': 1, 's[9]': 1}
{'s[0]': 0, 's[1]': 0, 's[2]': 0, 's[3]': 1, 's[4]': 0, 's[5]': 1, 's[6]': 1, 's[7]': 1, 's[8]': 1, 's[9]': 1}


In [47]:
for sample in response.samples():
    df=pd.DataFrame.from_dict(sample,orient='index').reset_index().reset_index()
    ans={'group1':{'list':list(df[df[0]==0]['level_0']),'sum':sum(list(df[df[0]==0]['level_0']))},'group2:':{'list':list(df[df[0]==1]['level_0']),'sum':sum(list(df[df[0]==0]['level_0']))}}
    print(pd.DataFrame.from_dict(ans))

               group1          group2:
list  [0, 1, 2, 3, 4]  [5, 6, 7, 8, 9]
sum                10               10
               group1          group2:
list  [0, 1, 2, 3, 4]  [5, 6, 7, 8, 9]
sum                10               10
               group1          group2:
list  [0, 1, 2, 3, 4]  [5, 6, 7, 8, 9]
sum                10               10
               group1          group2:
list  [0, 1, 2, 3, 4]  [5, 6, 7, 8, 9]
sum                10               10
            group1             group2:
list  [0, 1, 2, 4]  [3, 5, 6, 7, 8, 9]
sum              7                   7
